<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc-data-with-squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U tensorflow==2.0

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.2.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from pprint import pprint

In [0]:
project_path = "/content/drive/My Drive/GL-AIML/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"
datafile = project_path+'train-v2.0.json'

In [0]:
with open(datafile) as data_file:
        dataset_json = json.load(data_file)  #on trying to dataset_json.head(n=10),throwing error 'dict' object has no attribute 'head'
       # dataset = dataset_json["data"] #on trying to dataset.head(n=10),throwing error 'list' object has no attribute 'head'

  
  

In [6]:
# Explore JSON file

# Inspect Nested Keys
print('top-level-keys: {}'.format(list(dataset_json.keys())))
print('data keys: {}'.format(list(dataset_json['data'][0].keys())))
print('paragraphs keys: {}'.format(list(dataset_json['data'][0]['paragraphs'][0].keys())))
print('qas keys: {}'.format(list(dataset_json['data'][0]['paragraphs'][0]['qas'][0].keys())))
print('answers keys: {}'.format(list(dataset_json['data'][0]['paragraphs'][0]['qas'][0]['answers'][0].keys())))

top-level-keys: ['version', 'data']
data keys: ['title', 'paragraphs']
paragraphs keys: ['qas', 'context']
qas keys: ['question', 'id', 'answers', 'is_impossible']
answers keys: ['text', 'answer_start']


In [7]:
# Count Corpora
print('Nbr Corpora: {}'.format(len(dataset_json['data'])))

Nbr Corpora: 442


In [8]:
# Count Corpora
print(len(dataset_json['data']))

442


In [0]:
# Print Corpora Titles
pprint(list(pd.json_normalize(dataset_json,'data')['title']))

In [0]:
def convert_squad_to_tidy_df(json_dict, corpus):
    """This function converts the SQuAD JSON data to a Tidy Data Pandas Dataframe.
    
    :param obj json_dict: squad json data
    :param str corpus: name of squad corpora to select subset from json object
    
    :returns: converted json data
    :rtype: pandas dataframe
    
    """
    data = [c for c in json_dict['data'] if c['title']==corpus][0]
    df = pd.DataFrame()    
    data_paragraphs = data['paragraphs']
    for article_dict in data_paragraphs:
        row = []
        for answers_dict in article_dict['qas']:                    
          if answers_dict['is_impossible'] == True:
            for answer in answers_dict['plausible_answers']:
                row.append((c['title'],
                            article_dict['context'], 
                            answers_dict['question'], 
                            answers_dict['id'],
                            '',
                            '',                          
                            answer['answer_start'],
                            answer['text'],
                            answers_dict['is_impossible']
                           ))            
          else:
            for answer in answers_dict['answers']:
                row.append((c['title'],
                            article_dict['context'], 
                            answers_dict['question'], 
                            answers_dict['id'],
                            answer['answer_start'],
                            answer['text'],
                            '',
                            '',
                            answers_dict['is_impossible']
                           ))
        df = pd.concat([df, pd.DataFrame.from_records(row, columns=['title','context', 'question', 'id', 'answer_start', 'answer','plausible_answer_start', 'plausible_answer','is_impossible'])], axis=0, ignore_index=True)
        # df.drop_duplicates(inplace=True)
    return df


 

In [0]:
appended_data = []

lastindex = 0
for c in dataset_json['data']: 
  data = convert_squad_to_tidy_df(dataset_json, c['title'])
  # store DataFrame in list
  appended_data.append(data)

appended_data = pd.concat(appended_data)

In [0]:
appended_data.reset_index(drop=True,inplace = True)

In [0]:
appended_data['is_impossible'].value_counts()

False    86821
True     43498
Name: is_impossible, dtype: int64

In [51]:
appended_data[appended_data['id'] == '5a8d7bf7df8bba001a0f9ab1']

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible
2075,The_Legend_of_Zelda:_Twilight_Princess,The Legend of Zelda: Twilight Princess (Japane...,What category of game is Legend of Zelda: Aust...,5a8d7bf7df8bba001a0f9ab1,,,128,action-adventure,True


In [0]:
appended_data.to_csv( project_path + '/squad_train_data.csv')

# Possible EDAs:
 #Count Plot
 #Number of characters/words
 #Unique words
 #Word Cloud
 #Common Stopwords
 #Analysing punctuation
 #Ngram Analysis
 #Answer type distribution
 #Avg tokens/sentences
 #Comparison within datasets used


In [0]:
#1
#import seaborn as sns
#sns.countplot(data = appended_data, x = "context")
squad_df = pd.read_csv(project_path+'squad_train_data.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)

In [53]:
appended_data.tail()

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible
130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,,,485,matter,True
130315,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,,,327,Alfvén,True
130316,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,,,350,Gk. common matter,True
130317,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,,,529,a specifying modifier,True
130318,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,,,37,physics,True


In [0]:
# Features..
feature = ['context','question']

# Target
target ='answer'

In [56]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
# Prepare data

# Lemma

# stop word removal

# vectorization tfidvectoriation

# vectorization word2vec


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
# Stemming
#Function to Stem words
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

In [57]:
appended_data['question']

0                  When did Beyonce start becoming popular?
1         What areas did Beyonce compete in when she was...
2         When did Beyonce leave Destiny's Child and bec...
3             In what city and state did Beyonce  grow up? 
4                In which decade did Beyonce become famous?
                                ...                        
130314    Physics has broadly agreed on the definition o...
130315                 Who coined the term partonic matter?
130316                What is another name for anti-matter?
130317    Matter usually does not need to be used in con...
130318    What field of study has a variety of unusual c...
Name: question, Length: 130319, dtype: object

In [58]:
appended_data['context']

0         Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1         Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2         Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3         Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4         Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
                                ...                        
130314    The term "matter" is used throughout physics i...
130315    The term "matter" is used throughout physics i...
130316    The term "matter" is used throughout physics i...
130317    The term "matter" is used throughout physics i...
130318    The term "matter" is used throughout physics i...
Name: context, Length: 130319, dtype: object

In [59]:
appended_data.head()

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s,,,False
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing,,,False
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,526,2003,,,False
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,166,"Houston, Texas",,,False
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,276,late 1990s,,,False


In [0]:
X = appended_data[['context','question']].copy()

In [69]:
X.head()

,context,question
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?


In [0]:
y = appended_data[['answer']].copy()

In [0]:
# y[y['answer'] == ''][:] = 'IMPOSSIBLE'

y.loc[y['answer'] == '', 'answer'] = 'IMPOSSIBLE'

In [88]:
y

,answer
0,in the late 1990s
1,singing and dancing
2,2003
3,"Houston, Texas"
4,late 1990s
...,...
130314,IMPOSSIBLE
130315,IMPOSSIBLE
130316,IMPOSSIBLE
130317,IMPOSSIBLE


In [0]:
X.to_csv(project_path + '/X.csv')
y.to_csv(project_path + '/y.csv')

In [0]:
X = pd.read_csv(project_path + '/X.csv')
X.drop('Unnamed: 0',inplace=True,axis=1)
y = pd.read_csv(project_path + '/y.csv')
y.drop('Unnamed: 0',inplace=True,axis=1)

In [12]:
X.shape
y.shape

(130319, 1)

In [13]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

In [0]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=True):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)            
        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

In [95]:
remove_stopwords("sandipan laptop complex this is a nice day")

'sandipan laptop complex nice day'

In [16]:
print(preprocess_text(["sandipan UPPER laptop complex this is a nice day","abc def 123 $%% ddd", "   def 123"]))

['sandipan upper laptop complex nice day', 'abc def ddd', 'def']


In [0]:
X['clean_context'] = preprocess_text(X['context'])